# Bostrom supply simulation
---

## A/B simulations params (depricated)


| Param name | Simulation A | Simulation B|
|:---|---:|---:|
|Liquid Tokens (T)|200,000,000,000,000|200,000,000,000,000|
|Frozen Tokens (F)|700,000,000,000,000|700,000,000,000,000|
|Vested Tokens (L)|100,000,000,000,000|100,000,000,000,000|
|Initial inflation |5.00|5.00|
|Amper Mint Rate init|50.00|50.00|
|Volt Mint Rate init|100.00|100.00|
|Inflation rate change|13.00|13.00|
|Inflation min|1.00|1.00|
|Inflation max|15.00|15.00|
|Goal vested|80.00|80.00|
|Volt halving|1,000,000.00|1,000,000.00|
|Resource token init price|100,000,000.00|100,000,000.00|

In [ ]:
import time

# Standard libraries: https://docs.python.org/3/library/
import math

# Analysis and plotting modules
import pandas as pd

## Initial state

In [ ]:
simPeriod = 10          # amount of years for simulating
unitsPerYear = 365        # units per year(days)

In [ ]:
initial_state = {
    'T': 200_000_000_000_000,
    'F': 700_000_000_000_000,
    'L': 100_000_000_000_000,
    'A': 0,
    'V': 0,
    'I_r': 0.10,
    'd_l': 0,
    'MRa': 50,
    'MRv': 100,
    'CL': 0,
    'maxVestingTime': math.floor(unitsPerYear / 2),
    'agents_amount': 1000
}

In [ ]:
# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

from collections import Counter
from cadCAD import configs

# custom functions modules

from utils.psub import partial_state_update_blocks

## Params for simulating

In [ ]:
import itertools

vestingSpeed = [3, 6, 12, 18, 24, 30, 36]
unvestingSpeed = [72, 60, 48, 36, 30, 24]

parameter_sweep = list(itertools.product(vestingSpeed, unvestingSpeed))

vestingSpeed = [x[0] for x in parameter_sweep]
unvestingSpeed = [x[1] for x in parameter_sweep]

In [ ]:
system_params = {
    'inflationRateChange': [0.07], # maximum inflation rate change per year
    'inflationMin': [0.05], # minimum percent of the inflation
    'inflationMax': [0.20], # maximum percent of the inflation
    'goalVested': [0.70], # desireable ratio between vested tokens and tokens supply
    'unitsPerYear': [unitsPerYear], # units per year(minutes)
    'vestingSpeed': vestingSpeed, # amount of months to vest all liquid tokens
    'unvestingSpeed': unvestingSpeed, # amount of months to uninvest all vested tokens
    'voltHalving': [10_000_000], # 10 MegaVolt
    'MRa': [50], # initial mint rate for Amperes
    'MRv': [100], # initial mint rate for Voltes
    'base_vesting_resource': [100_000_000], # initial cost for resource token in 1 cycle
    'baseVestingTime': [math.floor(unitsPerYear / 12)],
    'boot_bonded': [0.7],
    'extra_links': [100],
    'guaranted_links': [100],
    'gift_activation': [math.floor(unitsPerYear / 3.65)],
    'start_agents_count': [1000],
    'capitalization_per_user': [1]
}

In [ ]:
del configs[:]
experiment = Experiment()

sim_config = config_sim({
    'N': 1,
    'T': range(int(math.ceil(simPeriod * unitsPerYear))),
    'M': system_params
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [ ]:
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
start_time = time.time()
simulation_result = pd.DataFrame(raw_result)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()

In [ ]:
df = simulation_result.copy()
df['Supply'] = df['T'] + df['F'] + df['L']
df['AmPrice'] = df['A'] / df['V']
df

In [ ]:
df

In [ ]:
# save local model file for power bi
# import pyarrow as pa
# import pyarrow.parquet as pq
# table = pa.Table.from_pandas(df)
# pq.write_table(table, 'simulation.parquet')

In [ ]:
from utils.plots import linear_plot, scatter_plot, df_preparator

In [ ]:
plot_df = df_preparator(df)

## Inflation

In [ ]:
scatter_plot(plot_df, "I_r", render='sns')

## Supply

In [ ]:
scatter_plot(plot_df, 'Supply', render='sns')

## Vested

In [ ]:
scatter_plot(plot_df, 'L', render='sns')

## Ampler tokens supply

In [ ]:
scatter_plot(plot_df, 'A', render='sns')

## Volt tokens supply

In [ ]:
scatter_plot(plot_df, 'V', render='sns')

## Mint Rate Volter-Amper

In [ ]:
# linear_plot(plot_df, ['MRa', 'MRv'], render='sns')

## Max vesting time

In [ ]:
linear_plot(plot_df, 'maxVestingTime', render='sns')

## Ratio between amper supply and volt supply

In [ ]:
scatter_plot(plot_df, 'AmPrice', render='sns')

## Cyberlinks

In [ ]:
scatter_plot(plot_df, 'CL', render='sns')

## Agents amount

In [ ]:
scatter_plot(plot_df, 'agents_amount', render='sns')

## Cap in ETH

In [ ]:
scatter_plot(plot_df, 'cap_in_eth', render='sns')

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))